### Indoor Scene Recognition

#### Import Modules

In [1]:
# IMPORT MODULES
# Import Basic Modules
import os
import shutil
import numpy as np
import pandas as pd
from skimage.io import imread
from skimage.transform import resize
import keras
from keras.utils import to_categorical
from keras.models import Sequential, Model, load_model
from keras.layers import Input, Add, Conv1D, Conv2D, MaxPooling1D, MaxPooling2D, Dense, Dropout, Activation, Flatten, ZeroPadding2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.initializers import glorot_uniform
from keras import regularizers
from sklearn.utils import shuffle
from sklearn.cross_validation import train_test_split
from keras.layers.advanced_activations import LeakyReLU
from keras.optimizers import Adam, SGD
import glob

Using TensorFlow backend.
/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:520: FutureWarn

#### Load Data

In [2]:
# LOAD DATA
# Change Directory
os.chdir('..')

In [24]:
# LOAD DATA
# Initialize
training_dir = 'Data/Images-Subset/'
images_dir = 'Data/Temp-Data/'
filenames = []
files = glob.glob('Data/Images-Subset/**/*.jpg')
labels = np.zeros((len(files), 1))
filenames_counter = 0
labels_counter = 0
print("Number of Files:", len(files))

Number of Files: 2647


In [22]:
import random
import string
def random_char(y):
       return ''.join(random.choice(string.ascii_letters) for x in range(y))


quiYG


In [25]:
# LOAD DATA
# Import Modules
import os
import os.path
from os import path
import shutil
import random
import string

# Define a Function to Generate Random ASCII Letters
def random_char(y):
       return ''.join(random.choice(string.ascii_letters) for x in range(y))

# Move Images
for root, dirs, files in os.walk((os.path.normpath(training_dir)), topdown=False):
        for name in files:
            counter = counter + 1
            if path.exists(name):
                os.rename(name, random_char(5) + name)   
            source_folder = os.path.join(root,name)
            shutil.copy2(source_folder, images_dir)

In [27]:
# LOAD DATA
# Assign Labels
for subdir, dirs, files in os.walk(training_dir, topdown=False):
    if (subdir == training_dir):
        continue
    for file in files:
        filenames.append(file), 
        labels[filenames_counter, 0] = labels_counter
        filenames_counter = filenames_counter + 1
    labels_counter = labels_counter + 1
    
    
print("Total Number of Files",len(filenames))
print("Total Number of Files Copied", len(glob.glob(images_dir + '*.jpg')))
print("Total Files Labelled", labels.shape)

Total Number of Files 2647
Total Number of Files Copied 2647
Total Files Labelled (2647, 1)


In [28]:
# LOAD DATA
# Create One-Hot Encoding
y_labels_one_hot = to_categorical(labels)

In [29]:
# LOAD DATA
# Shuffle Data
filenames_shuffled, y_labels_one_hot_shuffled = shuffle(filenames, y_labels_one_hot)

In [30]:
# LOAD DATA
# Create Training and Validation Datasets
filenames_shuffled_numpy = np.array(filenames_shuffled)
x_train_filenames, x_valid_filenames, y_train, y_valid = train_test_split(
    filenames_shuffled_numpy, y_labels_one_hot_shuffled, test_size=0.1, random_state=1)
print("Training Data Input Samples", x_train_filenames.shape)
print("Training Data Labels", y_train.shape)
print("Validation Data Input Samples", x_valid_filenames.shape)
print("Validation Data Labels", y_valid.shape)

Training Data Input Samples (2382,)
Training Data Labels (2382, 12)
Validation Data Input Samples (265,)
Validation Data Labels (265, 12)


In [32]:
# LOAD DATA
# Define a Class for Custom Data Generator
class CustomGenerator(keras.utils.Sequence): 
    # Define a Function for Initialization
    def __init__(self, image_filenames, labels, batch_size):
        self.image_filenames = image_filenames
        self.labels = labels
        self.batch_size = batch_size
    
    # Define a Function for Number of Batches
    def __len__(self):
        return (np.ceil(len(self.image_filenames) / float(self.batch_size))).astype(np.int)
  
    # Define a Function to Generate Batches
    def __getitem__(self, idx):
        batch_x = self.image_filenames[idx * self.batch_size:(idx+1) * self.batch_size]
        batch_y = self.labels[idx * self.batch_size:(idx+1) * self.batch_size]

        return (np.array([resize(imread(images_dir + str(filename)), (224, 224, 3))
                   for filename in batch_x])/255), np.array(batch_y)

In [33]:
# LOAD DATA
# Set Batch Size
batch_size = 32

# Create Training and Validation Batch Generators
training_batch_generator = CustomGenerator(x_train_filenames, y_train, batch_size)
validation_batch_generator = CustomGenerator(x_valid_filenames, y_valid, batch_size)

#### Model Architecture

In [34]:
# MODEL ARCHITECTURE
# Import Modules
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
from keras.applications.densenet import DenseNet121

# Define Model
def DenseNetModel(classes=12):
    # Create the Base Pre-Trained Model
    base_model = DenseNet121(weights='imagenet', include_top=False)

    # Add a Global Spatial Average Pooling Layer
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    
    # Add a Fully-Connected Layer
    x = Dense(1024, activation='relu')(x)
    
    # Add Logistic Layer for N Classes
    predictions = Dense(classes, activation='softmax')(x)

    # Create Resultant Model
    model = Model(inputs=base_model.input, outputs=predictions)
    
    return model

In [35]:
# MODEL ARCHITECTURE
# Create Model
model = DenseNetModel(classes=12)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, None, None, 3 0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, None, None, 6 9408        zero_padding2d_1[0][0]           
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, None, None, 6 256         conv1/conv[0][0]                 
__________________________________________________________________________________________________
conv1/relu

In [36]:
# MODEL ARCHITECTURE
# Compile Model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

#### Train

In [37]:
# TRAIN
# Train the Model
model.fit_generator(generator=training_batch_generator,
                   steps_per_epoch = int(len(x_train_filenames)/batch_size),
                   epochs = 5,
                   verbose = 1,
                   validation_data = validation_batch_generator,
                   validation_steps = int(len(x_valid_filenames)/batch_size))

Epoch 1/5
74/74 [==============================] - 679s 9s/step - loss: 1.6257 - acc: 0.4932 - val_loss: 2.2631 - val_acc: 0.3398
Epoch 2/5
74/74 [==============================] - 131s 2s/step - loss: 1.0541 - acc: 0.6573 - val_loss: 10.8132 - val_acc: 0.1094
Epoch 3/5
74/74 [==============================] - 89s 1s/step - loss: 0.7432 - acc: 0.7443 - val_loss: 3.0777 - val_acc: 0.3594
Epoch 4/5
74/74 [==============================] - 91s 1s/step - loss: 0.5800 - acc: 0.8068 - val_loss: 2.9800 - val_acc: 0.4258
Epoch 5/5
74/74 [==============================] - 103s 1s/step - loss: 0.4980 - acc: 0.8318 - val_loss: 2.5819 - val_acc: 0.5234
